In [1]:
import numpy as np
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
actions = ['best', 'hello', 'what', 'call', 'happy', 
           'iloveyou', 'see', 'smile', 'peace', 'me', 
           'meet', 'heart', 'fine']

data = np.concatenate([
    np.load('D:\mog_f\mog_dataset2\seq_best_1716781785.npy'),
    np.load('D:\mog_f\mog_dataset2\seq_hello_1716781785.npy'),
    np.load('D:\mog_f\mog_dataset2\seq_what_1716781785.npy'),
    np.load('D:\mog_f\mog_dataset2\seq_call_1716781785.npy'),
    np.load('D:\mog_f\mog_dataset2\seq_happy_1716781785.npy'),
    np.load('D:\mog_f\mog_dataset2\seq_iloveyou_1716781785.npy'),
    np.load('D:\mog_f\mog_dataset2\seq_see_1716781785.npy'),
    np.load('D:\mog_f\mog_dataset2\seq_smile_1716781785.npy'),
    np.load('D:\mog_f\mog_dataset2\seq_peace_1716781785.npy'),
    np.load('D:\mog_f\mog_dataset2\seq_me_1716781785.npy'),
    np.load('D:\mog_f\mog_dataset2\seq_meet_1716781785.npy'),
    np.load('D:\mog_f\mog_dataset2\seq_heart_1716781785.npy'),
    np.load('D:\mog_f\mog_dataset2\seq_fine_1716781785.npy')
], axis=0)

data.shape

(6714, 30, 100)

In [3]:
x_data = data[:, :, :-1]
labels = data[:, 0, -1]

print(x_data.shape)
print(labels.shape)

(6714, 30, 99)
(6714,)


In [4]:
from tensorflow.keras.utils import to_categorical

y_data = to_categorical(labels, num_classes=len(actions))
y_data.shape

(6714, 13)

In [6]:
from sklearn.model_selection import train_test_split

x_data = x_data.astype(np.float32)
y_data = y_data.astype(np.float32)

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=51)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

ModuleNotFoundError: No module named 'sklearn'

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(64, activation='relu', input_shape=x_train.shape[1:3]),
    Dense(32, activation='relu'),
    Dense(len(actions), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 64)             │        41,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 13)             │           429 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,493 (173.80 KB)

 Trainable params: 44,493 (173.80 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=200,
    callbacks=[
        ModelCheckpoint('models/mog-13-mediapipe.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=50, verbose=1, mode='auto')
    ]
)

In [12]:
from sklearn.metrics import multilabel_confusion_matrix
from tensorflow.keras.models import load_model

model = load_model('models/mog-13-mediapipe.h5')

y_pred = model.predict(x_val)

multilabel_confusion_matrix(np.argmax(y_val, axis=1), np.argmax(y_pred, axis=1))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


array([[[634,   0],
        [  0,  38]],

       [[592,   0],
        [  0,  80]],

       [[627,   0],
        [  0,  45]],

       [[638,   0],
        [  0,  34]],

       [[633,   0],
        [  0,  39]],

       [[632,   0],
        [  0,  40]],

       [[621,   0],
        [  0,  51]],

       [[632,   0],
        [  0,  40]],

       [[635,   0],
        [  0,  37]],

       [[627,   0],
        [  0,  45]],

       [[588,   0],
        [  0,  84]],

       [[587,   0],
        [  0,  85]],

       [[618,   0],
        [  0,  54]]], dtype=int64)

In [13]:
loss, accuracy = model.evaluate(x_val, y_val)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')

21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - acc: 1.0000 - loss: 0.0175
Validation Loss: 0.01645537093281746
Validation Accuracy: 1.0
